<a href="https://colab.research.google.com/github/arkeodev/langchain_tutorial/blob/main/YT_Exploring_ReAct_on_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia langchain-community langchain-openai

In [19]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()


··········


In [20]:
!pip show langchain

Name: langchain
Version: 0.2.12
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


## CoT - Chain of thought Reasoning

In [21]:
from langchain_openai.chat_models import ChatOpenAI

In [22]:
llm = ChatOpenAI(model_name='gpt-4o-mini',
             temperature=0,
             max_tokens = 256)

In [23]:
text = "Explain step by step. How old is the president of the United States?"

print(llm(text))

content='To determine the age of the President of the United States, you can follow these steps:\n\n1. **Identify the Current President**: As of October 2023, the President of the United States is Joe Biden.\n\n2. **Find the Birthdate**: Joe Biden was born on November 20, 1942.\n\n3. **Determine the Current Date**: The current date is October 2023.\n\n4. **Calculate the Age**:\n   - Start with the current year (2023) and subtract the birth year (1942):\n     \\[\n     2023 - 1942 = 81\n     \\]\n   - Since the current date (October) is before his birthday (November), you need to subtract one year from the total:\n     \\[\n     81 - 1 = 80\n     \\]\n\n5. **Conclusion**: As of October 2023, Joe Biden is 80 years old.\n\nIf you want to find the age of any future president, you would follow the same steps: identify the current president, find their birthdate, and calculate their age based on the current date.' response_metadata={'token_usage': {'completion_tokens': 232, 'prompt_tokens': 

In [24]:
text = "How would I get from Singapore to San Francisco?"

print(llm(text))

content='To travel from Singapore to San Francisco, you would typically follow these steps:\n\n1. **Book a Flight**: The most common way to travel between Singapore and San Francisco is by air. Look for flights from Singapore Changi Airport (SIN) to San Francisco International Airport (SFO). Several airlines operate on this route, including Singapore Airlines, United Airlines, and others. You may find direct flights or flights with one or more layovers.\n\n2. **Check Visa Requirements**: Depending on your nationality, you may need a visa to enter the United States. Check the U.S. Department of State website for visa requirements and apply in advance if necessary.\n\n3. **Prepare for the Flight**: Ensure you have all necessary travel documents, including your passport, visa (if required), and any COVID-19 related documentation (if applicable). Arrive at the airport at least 2-3 hours before your flight.\n\n4. **Travel to the Airport**: Use public transportation, a taxi, or a rideshare s

In [25]:
text = "Explain step by step. How would I get from Singapore to San Francisco?"

print(llm(text))

content='Traveling from Singapore to San Francisco involves several steps, including planning your trip, booking flights, and preparing for your arrival. Here’s a step-by-step guide:\n\n### Step 1: Research and Planning\n1. **Check Travel Restrictions**: Before planning your trip, check for any travel restrictions or requirements due to health regulations, visas, or other factors.\n2. **Decide on Travel Dates**: Determine when you want to travel. Consider factors like weather, events, and personal schedules.\n\n### Step 2: Book Your Flight\n1. **Search for Flights**: Use flight comparison websites (like Google Flights, Skyscanner, or Kayak) to find available flights from Singapore (SIN) to San Francisco (SFO).\n2. **Choose Your Flight**: Look for direct flights or flights with layovers. Direct flights are usually faster but may be more expensive.\n3. **Compare Prices**: Check different airlines and booking platforms to find the best price.\n4. **Book Your Ticket**: Once you find a suit

## Manual ReACT


In [26]:
question = "How old is the president of the United States?"

manual_react = f"""Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]
Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

Question: Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?
Thought: The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.
Action: Search[Milhouse]
Observation: Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.
Thought: The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".
Action: Lookup[named after]
Observation: (Result 1 / 1) Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.
Thought: Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.
Action: Finish[Richard Nixon]

Question: Which documentary is about Finnish rock groups, Adam Clayton Powell or The Saimaa Gesture?
Thought: I need to search Adam Clayton Powell and The Saimaa Gesture, and find which documentary is about Finnish rock groups.
Action: Search[Adam Clayton Powell]
Observation: Could not find [Adam Clayton Powell]. Similar: [’Adam Clayton Powell III’, ’Seventh Avenue (Manhattan)’, ’Adam Clayton Powell Jr. State Office Building’, ’Isabel Washington Powell’, ’Adam Powell’, ’Adam Clayton Powell (film)’, ’Giancarlo Esposito’].
Thought: To find the documentary, I can search Adam Clayton Powell (film).
Action: Search[Adam Clayton Powell (film)]
Observation: Adam Clayton Powell is a 1989 American documentary film directed by Richard Kilberg. The film is about the rise and fall of influential African-American politician Adam Clayton Powell Jr.[3][4] It was later aired as part of the PBS series The American Experience.
Thought: Adam Clayton Powell (film) is a documentary about an African-American politician, not Finnish rock groups. So the documentary about Finnish rock groups must instead be The Saimaa Gesture.
Action: Finish[The Saimaa Gesture]

Question: What profession does Nicholas Ray and Elia Kazan have in common?
Thought: I need to search Nicholas Ray and Elia Kazan, find their professions, then find the profession they have in common.
Action: Search[Nicholas Ray]
Observation: Nicholas Ray (born Raymond Nicholas Kienzle Jr., August 7, 1911 - June 16, 1979) was an American film director, screenwriter, and actor best known for the 1955 film Rebel Without a Cause.
Thought: Professions of Nicholas Ray are director, screenwriter, and actor. I need to search Elia Kazan next and find his professions.
Action: Search[Elia Kazan]
Observation: Elia Kazan was an American film and theatre director, producer, screenwriter and actor.
Thought: Professions of Elia Kazan are director, producer, screenwriter, and actor. So profession Nicholas Ray and Elia Kazan have in common is director, screenwriter, and actor.
Action: Finish[director, screenwriter, actor]

Question: Which magazine was started first Arthur’s Magazine or First for Women?
Thought: I need to search Arthur’s Magazine and First for Women, and find which was started first.
Action: Search[Arthur’s Magazine]
Observation: Arthur’s Magazine (1844-1846) was an American literary periodical published in Philadelphia in the 19th century.
Thought: Arthur’s Magazine was started in 1844. I need to search First for Women next.
Action: Search[First for Women]
Observation: First for Women is a woman’s magazine published by Bauer Media Group in the USA.[1] The magazine was started in 1989.
Thought: First for Women was started in 1989. 1844 (Arthur’s Magazine) < 1989 (First for Women), so Arthur’s Magazine was started first.
Action: Finish[Arthur’s Magazine]

Question:{question}"""

In [17]:
print(llm(manual_react))

content='To determine the age of the current president of the United States, I would need to know who the current president is and their birthdate. As of my last knowledge update in October 2023, Joe Biden is the president, born on November 20, 1942. \n\nTo calculate his age as of today (assuming today is October 2023), you would subtract his birth year from the current year:\n\n2023 - 1942 = 81 years old.\n\nHowever, since his birthday is in November, he has not yet turned 81 this year, so he is currently 80 years old. \n\nAction: Finish[80 years old]' response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 1163, 'total_tokens': 1296}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None} id='run-b2487b19-30b4-4368-aad1-435e47130d3a-0' usage_metadata={'input_tokens': 1163, 'output_tokens': 133, 'total_tokens': 1296}


## ReACT

“REACT: SYNERGIZING REASONING AND ACTING IN LANGUAGE MODELS” (Shunyu et al., 2022)

In [46]:
from langchain.prompts import PromptTemplate

template = """
    Answer the following questions as best you can. You have access to the following tools:

    {tools}

    Use the following format:

    Question: {input}
    Thought: Consider what information is needed and how to obtain it.
    Action: Choose one of the following actions [{tool_names}]
    Action Input: Provide the necessary input for the chosen action
    Observation: Record the result of the action

    Final Answer: {agent_scratchpad}
"""
prompt_template =  PromptTemplate(
    input_variables=["input", "tool_names", "tools", "agent_scratchpad"],
    template=template,
)

In [47]:
from langchain import Wikipedia
from langchain.agents import create_react_agent, Tool
from langchain.agents import AgentType, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer
from datetime import datetime

In [48]:
docstore=DocstoreExplorer(Wikipedia())

# Define the DateTime tool
date_time_tool = Tool(
    name="DateTime",
    func=lambda _: datetime.now().date(),
    description="Useful for getting the current date and time"
)

tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    ),
    date_time_tool
]

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template
)

# Initialize the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=5, return_intermediate_steps=True)

In [49]:
question = "How old is the president of the United States?"
response = agent_executor.invoke({"input": question})
print(response)



> Entering new AgentExecutor chain...
Question: How old is the president of the United States?  
Thought: I need to find out who the current president is and their date of birth to calculate their age.  
Action: Search  
Action Input: 'current president of the United States'  The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansions during the presidencies of Franklin D. Roosevelt and George W. Bush. In modern times, the president is o

In [50]:
question = "How old will the president of the United States be in June 2023?"
response = agent_executor.invoke({"input": question})
print(response)



> Entering new AgentExecutor chain...
Question: How old will the president of the United States be in June 2023?  
Thought: I need to know who the president of the United States is and their date of birth to calculate their age in June 2023.  
Action: Search  
Action Input: "current president of the United States"  The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansions during the presidencies of Franklin D. Roosevelt and George W. B

#### Let pull apart the Chain

In [51]:
# multi hop example
question = "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?"
response = agent_executor.invoke({"input": question})
print(response)



> Entering new AgentExecutor chain...
Question: What is the first film that Russell Crowe won an Oscar for, and who directed that movie?  
Thought: I need to find out which film Russell Crowe won his first Oscar for and the director of that film.  
Action: Search  
Action Input: 'first film Russell Crowe won Oscar for'  The Insider is a 1999 American biographical drama film directed by Michael Mann, from a screenplay adapted by Eric Roth and Mann based on Marie Brenner's 1996 Vanity Fair article "The Man Who Knew Too Much". The film stars Al Pacino, Russell Crowe, Christopher Plummer, Bruce McGill, Diane Venora, and Michael Gambon.
A fictionalized account of a true story, it centers on the controversial 60 Minutes segment about Jeffrey Wigand, a whistleblower in the tobacco industry, covering his and CBS producer Lowell Bergman's struggles as they defend his testimony against efforts to discredit and suppress it by CBS and Wigand's former employer.
Though not a box office success, Th

In [52]:
import langchain

langchain.debug = True

In [53]:
# multi hop example
question = "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?"
response = agent_executor.invoke({"input": question})
print(response)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the first film that Russell Crowe won an Oscar for, and who directed that movie?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:Runna

In [54]:
langchain.debug = False